In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
# optional: improves readibility by displaying all columns 
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:.2f}".format

# Load data

In [3]:
assessments = pd.read_csv("anonymisedData/assessments.csv")
courses = pd.read_csv("anonymisedData/courses.csv")
studentAssessment = pd.read_csv("anonymisedData/studentAssessment.csv")
studentInfo = pd.read_csv("anonymisedData/studentInfo.csv")
studentRegistration = pd.read_csv("anonymisedData/studentRegistration.csv")
studentVle = pd.read_csv("anonymisedData/studentVle.csv")
vle = pd.read_csv("anonymisedData/vle.csv")

## Duplicates

In [4]:
# Check for duplicates 
df_list = [assessments, courses, studentAssessment, studentInfo, studentRegistration, studentVle, vle]

for df in df_list:
    duplicates =  df[df.duplicated() == True]
    if len(duplicates) > 0:
          print(duplicates)

         code_module code_presentation  id_student  id_site  date  sum_click
2                AAA             2013J       28400   546652   -10          1
63               AAA             2013J       45462   546652   -10          1
180              AAA             2013J       77367   546652   -10          4
193              AAA             2013J       94961   546652   -10          2
442              AAA             2013J      248270   546652   -10          4
...              ...               ...         ...      ...   ...        ...
10655235         GGG             2014J      686194   897083   267          1
10655250         GGG             2014J      676071   896939   268          1
10655256         GGG             2014J      642905   896939   268          2
10655261         GGG             2014J      625530   896939   269          1
10655279         GGG             2014J      654064   896939   269          1

[787170 rows x 6 columns]


<span style="background-color: #BFE1BE"> 
    We are going to include this "duplicated" data since that they represent activity, and one row probably indicates 1 HTTP session. Student might have multiple similar sessions during a day.
</span>

## Table joins 
- student assessment data with assessment data (studentAssessment with assessments)
- student assessment data with course data (studentAssessment with courses)
- enrollment data with course data (studnentRegistration with courses)
- enrollment data with student data (studentRegistration with studentInfo)
- student activity data with online materials data (studentVle with vle)
- student activity data with course data (studentVle with courses)

<span style="background-color: #BFE1BE"> 
Create a common feature "course" to join tables
    </span>

In [5]:
df_list = [assessments, courses, studentAssessment, studentInfo, studentRegistration, studentVle, vle]

for df in df_list:
    if "code_module" in df.columns:
        df["course"] = df["code_module"] + "_" + df["code_presentation"]


In [6]:
studentAssessment = studentAssessment.merge(assessments, on="id_assessment", how="left")
studentAssessment = studentAssessment.merge(courses, on="course", how="left")
studentRegistration = studentRegistration.merge(courses, on=["course", "code_module", "code_presentation"], how="left")

<span style="background-color: #BFE1BE"> 
 Create a common feature "enrollment" to join tables

In [7]:
df_list = [assessments, courses, studentAssessment, studentInfo, studentRegistration, studentVle, vle]
for df in df_list:
    if ("id_student" in df.columns) & ("course" in df.columns):
        df["enrollment"] = df["id_student"].map(lambda x: str(x)) + "_" + df["course"]

In [8]:
studentRegistration = studentRegistration.merge(studentInfo, how="outer", on="enrollment", suffixes=('', '_y'))

# Drop duplicate columns
studentRegistration.drop(studentRegistration.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [9]:
studentVle = studentVle.merge(vle, how="left", on=["id_site", "course"])
studentVle = studentVle.merge(courses, on="course", how="outer")

## Students interaction table transformation and further joins

<span style="background-color: #BFE1BE"> 
To perform further joins we will create **pivot tables** that represent:
- **Click elements**: Sum of clicks per element per enrollment.
- **Click frequency**: Sum of clicks per course "quarter" per enrollment. Since courses have different duration, we will divide date columns into 4 bins, referring to each one as 1st/2nd/3rd/4th quarter. 

<span style="background-color: #BFE1BE"> 
Once tables are transformed we will perform **further joins**:
- enrollment data with click elements data (studentRegistration with click elements)
- enrollment data with click frequency data (studentRegistration with click frequency)

In [10]:
click_elements = studentVle.pivot_table(index="enrollment", columns="activity_type", values="sum_click", aggfunc=np.sum)
click_elements

activity_type,dataplus,dualpane,externalquiz,folder,forumng,glossary,homepage,htmlactivity,oucollaborate,oucontent,ouelluminate,ouwiki,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url
enrollment,,,,,,,,,,,,,,,,,,,,
100064_FFF_2013J,26.00,16.00,nan,6.00,567.00,nan,793.00,nan,3.00,3559.00,nan,59.00,25.00,34.00,705.00,nan,68.00,nan,590.00,63.00
100282_BBB_2013J,nan,nan,nan,nan,11.00,nan,25.00,nan,nan,nan,nan,nan,nan,nan,1.00,nan,1.00,nan,3.00,1.00
100561_DDD_2014J,nan,nan,32.00,nan,331.00,nan,222.00,nan,11.00,88.00,nan,nan,nan,nan,nan,nan,76.00,nan,158.00,32.00
100621_CCC_2014B,nan,nan,nan,nan,1.00,nan,30.00,nan,nan,8.00,nan,nan,nan,nan,1.00,nan,2.00,nan,16.00,1.00
1006742_FFF_2014B,nan,1.00,nan,nan,349.00,nan,526.00,nan,2.00,753.00,nan,121.00,16.00,9.00,195.00,nan,93.00,nan,310.00,46.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997948_DDD_2014B,nan,nan,nan,nan,21.00,nan,15.00,nan,nan,3.00,nan,nan,nan,nan,nan,nan,5.00,nan,17.00,nan
99799_BBB_2014B,nan,nan,nan,nan,116.00,7.00,182.00,nan,nan,7.00,nan,nan,nan,nan,80.00,nan,55.00,nan,90.00,16.00
998493_AAA_2014J,16.00,nan,nan,nan,5145.00,7.00,2821.00,nan,3.00,1911.00,nan,nan,nan,nan,nan,nan,29.00,nan,318.00,214.00


In [11]:
studentRegistration = studentRegistration.merge(click_elements, left_on="enrollment", right_index=True, how="outer")

In [12]:
# Create bins for date feature
studentVle["quarter"] = studentVle.apply(
                  lambda row: "clicks_before_start" if row["date"] < 0 else
                              "clicks_1st_quarter" if row["date"] < 0.25 * row["module_presentation_length"] else
                              "clicks_2nd_quarter" if row["date"] < 0.5 * row["module_presentation_length"] else
                              "clicks_3rd_quarter" if row["date"] < 0.75 * row["module_presentation_length"] else
                              "clicks_4th_quarter", axis=1)

In [13]:
click_frequency = studentVle.pivot_table(index="enrollment", columns="quarter", values="sum_click", aggfunc=np.sum)
click_frequency

quarter,clicks_1st_quarter,clicks_2nd_quarter,clicks_3rd_quarter,clicks_4th_quarter,clicks_before_start
enrollment,,,,,
100064_FFF_2013J,1133.00,1914.00,2310.00,1015.00,142.00
100282_BBB_2013J,42.00,nan,nan,nan,nan
100561_DDD_2014J,350.00,210.00,193.00,170.00,27.00
100621_CCC_2014B,5.00,nan,nan,nan,54.00
1006742_FFF_2014B,1857.00,15.00,nan,nan,549.00
...,...,...,...,...,...
997948_DDD_2014B,48.00,nan,nan,nan,13.00
99799_BBB_2014B,241.00,115.00,57.00,33.00,107.00
998493_AAA_2014J,4257.00,1706.00,1597.00,1757.00,1147.00


In [14]:
studentRegistration = studentRegistration.merge(click_frequency, left_on="enrollment", right_index=True, how="outer")

##  Student assessments and assessments tables transformation and further join

<span style="background-color: #BFE1BE">    
Handle date features (submition date and deadline). 
- Sanity checks: missing values, submition date earlier than start of the course. We will accept submition date bigger than the course lenght, since oftentimes these are accepted by professors. 
- Transform assessment submition date and deadline into **a binary feature that informs whether the submition was past deadline or not.**
 
<span style="background-color: #BFE1BE"> 
Handle assessment performance features (score, weights, banked?, past deadline?). We will treat exam and tests data separately, due to their different character.
- Exam data
    - Sanity checks: one exam records per enrollment and exam weight should be 100.
    - Join with enrollment data (studentRegistration).
- Tests data
    - Sanity checks: sum of weights per course should be 100.
    - Create **new features that summarize student performance during each enrollment: total graded (weighted) score, total ungraded score, total number of banked scores, total number of submitions past deadline.**


In [15]:
studentAssessment.describe()

,id_assessment,id_student,date_submitted,is_banked,score,date,weight,module_presentation_length
count,173912.00,173912.00,173912.00,173912.00,173739.00,171047.00,173912.00,173912.00
mean,26553.80,705150.72,116.03,0.01,75.80,130.61,12.74,255.23
std,8829.78,552395.19,71.48,0.10,18.80,78.03,17.88,13.58
min,1752.00,6516.00,-11.00,0.00,0.00,12.00,0.00,234.00
25%,15022.00,504429.00,51.00,0.00,65.00,54.00,0.00,240.00
50%,25359.00,585208.00,116.00,0.00,80.00,129.00,9.00,262.00
75%,34883.00,634498.00,173.00,0.00,90.00,214.00,18.00,268.00
max,37443.00,2698588.00,608.00,1.00,100.00,261.00,100.00,269.00


In [16]:
studentAssessment.isnull().sum()

id_assessment                    0
id_student                       0
date_submitted                   0
is_banked                        0
score                          173
code_module_x                    0
code_presentation_x              0
assessment_type                  0
date                          2865
weight                           0
course                           0
code_module_y                    0
code_presentation_y              0
module_presentation_length       0
enrollment                       0
dtype: int64

<span style="background-color: #BFE1BE"> 
Date features

We know that the deadline "date" for the final exam is the last day of the course. Let's see how many records with missing deadline information refer to the final exam.

In [17]:
studentAssessment["assessment_type"][studentAssessment["date"].isnull() == True].value_counts()

Exam    2865
Name: assessment_type, dtype: int64

Fortunately, all records refer to final exam. 

In [18]:
# Fill missing values with the last day of the course
studentAssessment['date'].fillna(value=studentAssessment["module_presentation_length"], inplace=True)

In [19]:
# Create a feature "past_deadline"
studentAssessment["past_deadline"] = (studentAssessment["date_submitted"] > studentAssessment["date"]).astype(int)

<span style="background-color: #BFE1BE"> 
Assessment performance features
- Exams

In [20]:
# One final exam record per enrollment
filter_exams = studentAssessment["assessment_type"] == "Exam"
assert len(studentAssessment.loc[filter_exams]) == len(np.unique(studentAssessment.loc[filter_exams]["enrollment"]))

In [21]:
# All exams have weight equal to 100
assert (studentAssessment["weight"].loc[filter_exams] != 100).sum() == 0

In [22]:
exam_data_to_merge = studentAssessment.loc[filter_exams][["score", "past_deadline", "enrollment"]]
studentRegistration = studentRegistration.merge(exam_data_to_merge, on="enrollment", how="outer")

<span style="background-color: #BFE1BE"> 
Assessment performance features
- Tests

In [23]:
filter_tests = assessments["assessment_type"] != "Exam"

We can see that some exams have a zero weight. Before we create a new feature "weighted_score" let's investigate if zero weights are correct. From the domain knowledge we know that all courses should include tests which weights sum to 100.

In [24]:
# Verify if the sum of tests weights per course is indeed 100
assessments[["course", "weight"]][filter_tests].groupby(["course"]).sum()

,weight
course,
AAA_2013J,100.00
AAA_2014J,100.00
BBB_2013B,100.00
BBB_2013J,100.00
BBB_2014B,100.00
BBB_2014J,100.00
CCC_2014B,100.00
CCC_2014J,100.00
DDD_2013B,100.00


Courses GGG_2013J, GGG_2014B and GGG_2014J do not have weighted tests. We will impute the mean weight calculated per test assessment type (CMA / TMA) and by scaling newly imputed values we will make sure their sum gives 100.

In [25]:
# Mean weigth per test type
filter_non_0_weights = assessments["weight"] > 0.
mean_weights = assessments[["assessment_type", "weight"]][filter_non_0_weights].groupby('assessment_type').agg("mean")
mean_weights

,weight
assessment_type,
CMA,3.00
Exam,100.00
TMA,18.85


In [26]:
# Impute
assessments["weight"] = assessments.apply(
                lambda row: mean_weights.loc[row["assessment_type"], :][0]
                if ((row["course"] in ["GGG_2013J", "GGG_2014B", "GGG_2014J"]) & (row["assessment_type"] != "Exam"))
                else row["weight"], axis=1)       

In [27]:
# Check sum of imputed weights per course
GGG_courses = assessments["course"].isin(["GGG_2013J", "GGG_2014B", "GGG_2014J"])
assessments[["course", "weight"]][GGG_courses].groupby(["course"]).sum()

,weight
course,
GGG_2013J,174.56
GGG_2014B,174.56
GGG_2014J,174.56


In [28]:
# Scale weights so that the sum is 100
assessments["weight"] = assessments.apply(
                lambda row: 100. * row["weight"] / 74.56
                if (row["course"] in str(["GGG_2013J", "GGG_2014B", "GGG_2014J"])) & (row["assessment_type"] != "Exam")
                else row["weight"], axis=1)    

In [29]:
# Sanity check: sum of weights
assessments[["course", "weight"]][filter_tests & GGG_courses].groupby(["course"]).sum()

,weight
course,
GGG_2013J,100.00
GGG_2014B,100.00
GGG_2014J,100.00


In [30]:
# Update test weights in studentAssessment table
studentAssessment.drop("weight", axis=1, inplace=True)
studentAssessment = studentAssessment.merge(assessments[["id_assessment", "weight"]] , on="id_assessment")

In [31]:
# Create weighted score
studentAssessment["weighted_score"] = studentAssessment["score"] * studentAssessment["weight"] / 100.

We will now calculate the number of graded tests taken per enrollment.

In [32]:
# Filter graded tests
graded_tests = (studentAssessment["assessment_type"] != "Exam") & (studentAssessment["weight"] > 0.)

# Aggregate weighted scores, ungraded scores, is_banked, past_deadline for each enrollment
tests_per_enrollment = studentAssessment.loc[graded_tests].groupby("enrollment").agg('sum')

In [33]:
tests_per_enrollment = tests_per_enrollment.loc[:, ["past_deadline", "weighted_score", "weight", "is_banked"]]
tests_per_enrollment.columns = ["num_past_deadline", "total_weighted_score", "total_weights", "num_banked"]
tests_per_enrollment

,num_past_deadline,total_weighted_score,total_weights,num_banked
enrollment,,,,
100064_FFF_2013J,2,92.00,100.00,0
100282_BBB_2013J,0,3.90,5.00,0
100561_DDD_2014J,0,72.50,100.00,0
1006742_FFF_2014B,0,9.75,12.50,0
100788_CCC_2014J,4,21.93,25.00,0
...,...,...,...,...
99670_FFF_2014B,1,89.50,100.00,0
99799_BBB_2014B,6,62.66,82.00,0
998493_AAA_2014J,0,67.60,100.00,0


In [34]:
df = studentRegistration.merge(tests_per_enrollment, left_on="enrollment", right_index=True, how="outer")

In [38]:
# Move id column to the very left 
enrollment = df["enrollment"]
df.insert(0, "enrollment_id", enrollment)
df.drop("enrollment", axis=1, inplace=True)

In [42]:
df.to_csv("data.csv")